In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 동별 객단가 예측

In [9]:
import os
import joblib
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta

# =========================
# 설정
# =========================
from dotenv import load_dotenv
load_dotenv()

SERVICE_KEY = os.getenv("RAIN_ID")  # 기상청 단기예보 서비스키

MODELS_ROOT = "data/model"
BEST_MAP_PATH = os.path.join(MODELS_ROOT, "results_by_hour.csv")
BUNDLE_PATH_TEMPLATE = os.path.join(MODELS_ROOT, "HOUR_{hour:02d}", "best_model.joblib")

# 학습 때 로그타겟이면 True
USE_LOG_TARGET = False

# (학습 때와 동일해야 함) 안전장치로 drop
DROP_COLS = ["UNIT", "DATE", "AMT", "CNT"]

# 격자 엑셀
GRID_XLSX_PATH = "data/수원시 격자.xlsx"
_GRID_CACHE = None  # 반복 로딩 방지 캐시

# =========================
# 항목요약 시간대(01~10)
# =========================
HOUR_BINS = {
    1: list(range(0, 7)),     # 00~06
    2: [7, 8],
    3: [9, 10],
    4: [11, 12],
    5: [13, 14],
    6: [15, 16],
    7: [17, 18],
    8: [19, 20],
    9: [21, 22],
    10: [23],
}

# =========================
# 날짜 파생 + DAY(01=월..07=일)
# =========================
def compute_day_code(date_yyyymmdd: str) -> int:
    dt = pd.to_datetime(str(date_yyyymmdd), format="%Y%m%d", errors="raise")
    return int(dt.weekday() + 1)

def make_date_features(date_yyyymmdd: str) -> dict:
    dt = pd.to_datetime(str(date_yyyymmdd), format="%Y%m%d", errors="raise")
    return {
        "TA_YMD": str(date_yyyymmdd),
        "YEAR": int(dt.year),
        "MONTH": int(dt.month),
        "DAY_OF_MONTH": int(dt.day),
        "WEEKOFYEAR": int(dt.isocalendar().week),
    }

# =========================
# 격자: DONG -> (nx, ny)
# =========================
def load_grid_table(path: str = GRID_XLSX_PATH) -> pd.DataFrame:
    """
    data/수원시 격자.xlsx
    - A열: 동이름, B열: nx, C열: ny (사용자 설명 기준)
    - header가 없다고 가정하고 A/B/C를 고정으로 읽음
      (만약 실제 파일에 헤더가 있으면 header=None -> header=0 으로 변경)
    """
    global _GRID_CACHE
    if _GRID_CACHE is not None:
        return _GRID_CACHE

    if not os.path.exists(path):
        raise FileNotFoundError(f"격자 파일이 없습니다: {path}")

    df = pd.read_excel(path, header=None)
    if df.shape[1] < 3:
        raise ValueError(f"격자 파일 컬럼 수가 부족합니다(A,B,C 필요): {path}")

    df = df.iloc[:, :3].copy()
    df.columns = ["DONG_NAME", "nx", "ny"]

    df["DONG_NAME"] = df["DONG_NAME"].astype(str).str.strip()
    df["nx"] = pd.to_numeric(df["nx"], errors="coerce")
    df["ny"] = pd.to_numeric(df["ny"], errors="coerce")
    df = df.dropna(subset=["nx", "ny"]).copy()
    df["nx"] = df["nx"].astype(int)
    df["ny"] = df["ny"].astype(int)

    _GRID_CACHE = df
    return df

def find_grid_by_dong(dong: str, path: str = GRID_XLSX_PATH) -> tuple[int, int]:
    """
    사용자가 입력한 dong(예: '행궁동', '수원시 팔달구 행궁동')으로
    엑셀 A열(DONG_NAME)에서 격자(nx,ny) 찾기.
    - 완전일치 우선, 없으면 포함검색(양방향)
    - 후보가 여러 개면 DONG_NAME 길이가 짧은 것을 우선 선택
    """
    dong = str(dong).strip()
    if not dong:
        raise ValueError("dong(동) 입력이 비었습니다.")

    grid = load_grid_table(path)

    # 1) 완전일치
    exact = grid[grid["DONG_NAME"] == dong]
    if not exact.empty:
        r = exact.iloc[0]
        return int(r["nx"]), int(r["ny"])

    # 2) 포함검색(양방향)
    a = grid["DONG_NAME"].str.contains(dong, na=False)
    b = grid["DONG_NAME"].apply(lambda x: str(dong).find(str(x)) >= 0)
    cand = grid[a | b].copy()

    # fallback: 마지막 토큰(보통 'OO동')로 재시도
    if cand.empty:
        tok = dong.split()[-1]
        cand = grid[grid["DONG_NAME"].str.contains(tok, na=False)].copy()

    if cand.empty:
        raise ValueError(f"'{dong}'에 해당하는 격자(nx,ny)를 찾지 못했습니다. 엑셀 A열 동이름을 확인하세요.")

    cand["name_len"] = cand["DONG_NAME"].str.len()
    cand = cand.sort_values(["name_len"]).reset_index(drop=True)
    r = cand.iloc[0]
    return int(r["nx"]), int(r["ny"])

# =========================
# 기상청 단기예보 TMP/PCP
# =========================
def _parse_pcp_to_mm(x):
    if x is None:
        return 0.0
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip()
    if s in ("강수없음", "없음", "", "nan", "NaN"):
        return 0.0
    if "미만" in s:
        try:
            num = float(s.replace("mm", "").replace("미만", "").strip())
            return max(0.0, num * 0.5)
        except:
            return 0.0
    if "~" in s:
        try:
            a, b = s.replace("mm", "").split("~")
            return (float(a) + float(b)) / 2.0
        except:
            return 0.0
    try:
        return float(s.replace("mm", ""))
    except:
        return 0.0

def fetch_vilage_fcst_json(service_key, base_date, base_time, nx, ny, num_rows=3000):
    url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    params = {
        "serviceKey": service_key,
        "pageNo": 1,
        "numOfRows": num_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def fetch_hourly_tmp_pcp_for_date(service_key, target_yyyymmdd, nx, ny):
    base_times = ["2300", "2000", "1700", "1400", "1100", "0800", "0500", "0200"]
    now = datetime.now()
    today = now.strftime("%Y%m%d")
    yesterday = (now - timedelta(days=1)).strftime("%Y%m%d")

    last_err = None
    for bd in [today, yesterday]:
        for bt in base_times:
            try:
                js = fetch_vilage_fcst_json(service_key, bd, bt, nx, ny)
                items = js["response"]["body"]["items"]["item"]
                df = pd.DataFrame(items)

                df = df[df["category"].isin(["TMP", "PCP"])]
                df = df[df["fcstDate"].astype(str) == str(target_yyyymmdd)]
                if df.empty:
                    continue

                df["fcstTime"] = df["fcstTime"].astype(str).str.zfill(4)
                piv = df.pivot_table(
                    index="fcstTime",
                    columns="category",
                    values="fcstValue",
                    aggfunc="first"
                ).reset_index()

                if "TMP" not in piv.columns:
                    continue

                piv["TMP"] = piv["TMP"].astype(float)
                piv["PCP"] = piv["PCP"].apply(_parse_pcp_to_mm) if "PCP" in piv.columns else 0.0
                piv["PCP"] = piv["PCP"].astype(float)
                piv["base_used"] = f"{bd}-{bt}"
                return piv.sort_values("fcstTime").reset_index(drop=True)
            except Exception as e:
                last_err = e
                continue

    raise RuntimeError(f"해당 날짜의 기상청 예보 데이터를 찾지 못했습니다. last_err={last_err}")

def aggregate_weather_to_item_slots(hourly_df: pd.DataFrame) -> dict:
    hourly_df = hourly_df.copy()
    hourly_df["HOUR_OF_DAY"] = hourly_df["fcstTime"].str[:2].astype(int)

    out = {}
    for h, hours in HOUR_BINS.items():
        sub = hourly_df[hourly_df["HOUR_OF_DAY"].isin(hours)]
        out[h] = {
            "TEMP": float(sub["TMP"].mean()) if not sub.empty else np.nan,
            "RAIN": float(sub["PCP"].sum()) if not sub.empty else 0.0,
        }

    temps = [v["TEMP"] for v in out.values() if not pd.isna(v["TEMP"])]
    fill_temp = float(np.mean(temps)) if temps else 0.0
    for h in out:
        if pd.isna(out[h]["TEMP"]):
            out[h]["TEMP"] = fill_temp

    return out

# =========================
# ✅ 동 입력 -> 그 동의 "기온" 반환(원하는 함수)
# =========================
def get_temperature_by_dong(
    date_yyyymmdd: str,
    dong: str,
    slot_hour: int = 5,
    service_key: str = SERVICE_KEY
) -> float:
    """
    dong 입력 -> 격자(nx,ny) 조회 -> 기상청 예보 -> slot_hour(1~10)의 TEMP 반환
    """
    if not service_key:
        raise ValueError("SERVICE_KEY(RAIN_ID)가 비어있습니다. .env 또는 환경변수를 확인하세요.")
    if not (1 <= int(slot_hour) <= 10):
        raise ValueError("slot_hour는 1~10 이어야 합니다.")

    nx, ny = find_grid_by_dong(dong)
    hourly_weather = fetch_hourly_tmp_pcp_for_date(service_key, date_yyyymmdd, nx, ny)
    weather_by_hour = aggregate_weather_to_item_slots(hourly_weather)

    temp = float(weather_by_hour[int(slot_hour)]["TEMP"])
    return round(temp, 1)  # ✅ 소수점 1자리 반환

# =========================
# 모델 로드 + "모델이 기대하는 컬럼" 자동 맞춤
# (학습 저장물: best_model.joblib = {"preprocess": ColumnTransformer, "model": estimator, "model_name": str})
# =========================
def load_best_model_map():
    if not os.path.exists(BEST_MAP_PATH):
        raise FileNotFoundError(f"결과 파일이 없습니다: {BEST_MAP_PATH}")
    df = pd.read_csv(BEST_MAP_PATH)
    required_cols = {"HOUR", "BEST_MODEL"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"{BEST_MAP_PATH}에 필요한 컬럼이 없습니다. 필요: {required_cols}, 현재: {set(df.columns)}")
    return df.set_index("HOUR")["BEST_MODEL"].to_dict()

def load_best_bundle(hour: int):
    path = BUNDLE_PATH_TEMPLATE.format(hour=hour)
    if not os.path.exists(path):
        raise FileNotFoundError(f"번들 파일이 없습니다: {path}")
    bundle = joblib.load(path)
    if not isinstance(bundle, dict) or "preprocess" not in bundle or "model" not in bundle:
        raise ValueError(f"번들 포맷이 예상과 다릅니다: {path}")
    return bundle

def get_required_feature_names_from_preprocess(preprocess) -> list:
    names = []
    for name, trans, cols in getattr(preprocess, "transformers_", []):
        if cols is None:
            continue
        if isinstance(cols, (list, tuple, np.ndarray, pd.Index)):
            names.extend(list(cols))
    return sorted(set([str(x) for x in names]))

def align_row_to_required_columns(X_row: pd.DataFrame, required_cols: list) -> pd.DataFrame:
    X = X_row.copy()

    if required_cols:
        drop_cols = [c for c in X.columns if c not in required_cols]
        if drop_cols:
            X = X.drop(columns=drop_cols, errors="ignore")

        for c in required_cols:
            if c not in X.columns:
                X[c] = np.nan

        X = X[required_cols]

    return X

def predict_with_bundle(bundle: dict, X_new: pd.DataFrame) -> np.ndarray:
    pre = bundle["preprocess"]
    model = bundle["model"]
    X_enc = pre.transform(X_new)
    return model.predict(X_enc)

# =========================
# 핵심: 날짜 1개 → 1~10시간대 예측
# - (nx, ny 직접 입력 버전)
# =========================
def predict_date_all_hours_with_weather(
    date_yyyymmdd: str,
    nx: int,
    ny: int,
    service_key: str = SERVICE_KEY
) -> pd.DataFrame:
    if not service_key:
        raise ValueError("SERVICE_KEY(RAIN_ID)가 비어있습니다. .env 또는 환경변수를 확인하세요.")

    day_code = compute_day_code(date_yyyymmdd)

    hourly_weather = fetch_hourly_tmp_pcp_for_date(service_key, date_yyyymmdd, nx, ny)
    weather_by_hour = aggregate_weather_to_item_slots(hourly_weather)

    best_map = load_best_model_map()
    rows = []

    for hour in range(1, 11):
        if hour not in best_map:
            continue

        bundle = load_best_bundle(hour)
        best_name = str(best_map[hour])

        row = {}
        row.update(make_date_features(date_yyyymmdd))
        row.update({
            "HOUR": int(hour),
            "DAY": int(day_code),
            "TEMP": float(weather_by_hour[hour]["TEMP"]),
            "RAIN": float(weather_by_hour[hour]["RAIN"]),
            "DELIV_TEMP": float(weather_by_hour[hour]["TEMP"]),
            "DELIV_RAIN": float(weather_by_hour[hour]["RAIN"]),
        })

        X_row = pd.DataFrame([row])
        X_row = X_row.drop(columns=[c for c in DROP_COLS if c in X_row.columns], errors="ignore")

        required = get_required_feature_names_from_preprocess(bundle["preprocess"])
        X_row_aligned = align_row_to_required_columns(X_row, required)

        pred = float(predict_with_bundle(bundle, X_row_aligned)[0])
        if USE_LOG_TARGET:
            pred = float(np.expm1(pred))

        rows.append({
            "TA_YMD": str(date_yyyymmdd),
            "DAY": int(day_code),
            "HOUR": int(hour),
            "BEST_MODEL": best_name,
            "TEMP": round(float(row["TEMP"]), 2), 
            "RAIN": float(row["RAIN"]),
            "PRED_UNIT": int(round(pred)),
            "KMA_BASE_USED": str(hourly_weather.loc[0, "base_used"]),
        })

    return pd.DataFrame(rows).sort_values("HOUR").reset_index(drop=True)

# =========================
# ✅ 핵심: 사용자 DONG 입력 버전 (요청한 흐름)
# - DONG으로 nx,ny 자동 추출
# - 그 nx,ny로 예측 수행
# =========================
def predict_date_all_hours_with_weather_by_dong(
    date_yyyymmdd: str,
    dong: str,
    service_key: str = SERVICE_KEY
) -> pd.DataFrame:
    nx, ny = find_grid_by_dong(dong)
    df_pred = predict_date_all_hours_with_weather(
        date_yyyymmdd=date_yyyymmdd,
        nx=nx,
        ny=ny,
        service_key=service_key
    )
    df_pred.insert(1, "DONG", str(dong))
#     df_pred.insert(2, "nx", int(nx))
#     df_pred.insert(3, "ny", int(ny))
    return df_pred

# =========================
# 사용 예시
# =========================
if __name__ == "__main__":
    # 1) 동의 특정 시간대(항목요약) 기온만 뽑기
#     temp = get_temperature_by_dong("20251231", dong="정자2동", slot_hour=5)
#     print("TEMP(slot_hour=5):", temp, "°C")

    # 2) 동 입력으로 1~10 시간대 매출(UNIT) 예측
    df_pred = predict_date_all_hours_with_weather_by_dong(
        date_yyyymmdd="20251231",
        dong="행궁동"
    )
    print(df_pred)


     TA_YMD DONG  DAY  HOUR BEST_MODEL   TEMP  RAIN  PRED_UNIT  KMA_BASE_USED
0  20251231  행궁동    3     1        XGB  -6.43   0.0      36440  20251230-1100
1  20251231  행궁동    3     2        XGB  -8.00   0.0      11853  20251230-1100
2  20251231  행궁동    3     3        XGB  -7.50   0.0      16005  20251230-1100
3  20251231  행궁동    3     4  EXTRATREE  -5.50   0.0      24844  20251230-1100
4  20251231  행궁동    3     5  EXTRATREE  -5.00   0.0      25553  20251230-1100
5  20251231  행궁동    3     6        XGB  -5.00   0.0      21377  20251230-1100
6  20251231  행궁동    3     7  EXTRATREE  -6.00   0.0      26994  20251230-1100
7  20251231  행궁동    3     8  EXTRATREE  -7.50   0.0      45340  20251230-1100
8  20251231  행궁동    3     9        XGB  -8.50   0.0      48005  20251230-1100
9  20251231  행궁동    3    10        XGB -10.00   0.0      43143  20251230-1100


# 동별 매출 예측

In [17]:
"""
시간대별 AMT/CNT 예측 (수원시 한식 동별 데이터)
- 입력: 날짜(TA_YMD), DONG, (기상청 API로 가져온) 시간대별 TEMP/RAIN
- 출력: 시간대별 AMT, CNT
"""

import os
import joblib
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from dotenv import load_dotenv

# =========================
# A) 설정
# =========================
load_dotenv('')
SERVICE_KEY = os.getenv('RAIN_ID')

DATA_CSV = "data/수원시 한식 동별 데이터백업.csv"
MODEL_DIR = "data/models_hourly_amt_cnt"
os.makedirs(MODEL_DIR, exist_ok=True)

# ✅ 예측 결과 저장 폴더(명시)
OUTPUT_DIR = "data/pred_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

KMA_VILAGE_URL = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
BASE_TIMES = ["2300", "2000", "1700", "1400", "1100", "0800", "0500", "0200"]

HOUR_SLOT_TO_REP_TIME = {
    1: "0300",
    2: "0800",
    3: "1000",
    4: "1200",
    5: "1400",
    6: "1600",
    7: "1800",
    8: "2000",
    9: "2200",
    10: "2300",
}

DONG_GRID_XLSX = "data/수원시 격자.xlsx"
_DONG_TO_GRID = None  # lazy cache


# =========================
# B) 유틸: 요일 계산 (1~7, 월=1 ... 일=7)
# =========================
def ymd_to_day_1to7(yyyymmdd: str) -> int:
    dt = datetime.strptime(str(yyyymmdd), "%Y%m%d")
    return dt.weekday() + 1


# =========================
# DONG -> (nx,ny) 로딩/조회
# =========================
def load_dong_to_grid(xlsx_path: str = DONG_GRID_XLSX) -> dict:
    global _DONG_TO_GRID
    if _DONG_TO_GRID is not None:
        return _DONG_TO_GRID

    if not os.path.exists(xlsx_path):
        raise FileNotFoundError(f"격자 파일이 없습니다: {xlsx_path}")

    df = pd.read_excel(xlsx_path, header=None)
    if df.shape[1] < 3:
        raise ValueError(f"격자 파일 컬럼 수가 부족합니다(A,B,C 필요): {xlsx_path}")

    df = df.iloc[:, :3].copy()
    df.columns = ["DONG_NAME", "nx", "ny"]

    df["DONG_NAME"] = df["DONG_NAME"].astype(str).str.strip()
    df["nx"] = pd.to_numeric(df["nx"], errors="coerce")
    df["ny"] = pd.to_numeric(df["ny"], errors="coerce")
    df = df.dropna(subset=["nx", "ny"]).copy()
    df["nx"] = df["nx"].astype(int)
    df["ny"] = df["ny"].astype(int)

    _DONG_TO_GRID = {row["DONG_NAME"]: (int(row["nx"]), int(row["ny"])) for _, row in df.iterrows()}
    return _DONG_TO_GRID


def find_grid_by_dong(dong: str, xlsx_path: str = DONG_GRID_XLSX) -> tuple[int, int]:
    dong = str(dong).strip()
    if not dong:
        raise ValueError("dong 입력이 비었습니다.")

    dmap = load_dong_to_grid(xlsx_path)

    if dong in dmap:
        return dmap[dong]

    keys = list(dmap.keys())
    cand = []
    for k in keys:
        if (k in dong) or (dong in k):
            cand.append(k)

    if not cand:
        tok = dong.split()[-1]
        for k in keys:
            if tok in k or k in tok:
                cand.append(k)

    if not cand:
        raise ValueError(f"'{dong}'에 해당하는 nx/ny를 격자 엑셀에서 찾지 못했습니다. A열 동이름을 확인하세요.")

    cand = sorted(cand, key=len)
    return dmap[cand[0]]


# =========================
# C) 기상청 API: base_date/base_time 결정
# =========================
def pick_base_datetime_kst(now_kst: datetime) -> tuple[str, str]:
    today = now_kst.strftime("%Y%m%d")
    hhmm = now_kst.strftime("%H%M")

    for bt in BASE_TIMES:
        if hhmm >= bt:
            return today, bt

    yday = (now_kst - timedelta(days=1)).strftime("%Y%m%d")
    return yday, "2300"


# =========================
# D) 기상청 단기예보 호출 + 특정 날짜/시각 TMP, PCP 추출
# =========================
def fetch_vilage_fcst_items(service_key: str, base_date: str, base_time: str, nx: int, ny: int,
                            num_of_rows: int = 2000) -> list[dict]:
    params = {
        "serviceKey": service_key,
        "pageNo": 1,
        "numOfRows": num_of_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }
    r = requests.get(KMA_VILAGE_URL, params=params, timeout=20)
    r.raise_for_status()
    data = r.json()
    return data["response"]["body"]["items"]["item"]


def parse_pcp_to_float(pcp_val) -> float:
    if pcp_val is None:
        return 0.0
    s = str(pcp_val).strip()
    if s in ("강수없음", "없음", ""):
        return 0.0
    s = s.replace("mm", "").replace("미만", "").strip()
    try:
        return float(s)
    except:
        return 0.0


def get_temp_rain_for_datetime(service_key: str, dong: str, target_yyyymmdd: str, target_hhmm: str) -> tuple[float, float]:
    nx, ny = find_grid_by_dong(dong)

    now_kst = datetime.now(timezone(timedelta(hours=9)))
    base_date, base_time = pick_base_datetime_kst(now_kst)

    items = fetch_vilage_fcst_items(service_key, base_date, base_time, nx, ny)

    tmp = None
    pcp = None
    for it in items:
        if it.get("fcstDate") == target_yyyymmdd and it.get("fcstTime") == target_hhmm:
            cat = it.get("category")
            if cat == "TMP":
                tmp = float(it.get("fcstValue"))
            elif cat == "PCP":
                pcp = parse_pcp_to_float(it.get("fcstValue"))

    if tmp is None:
        tmp = 0.0
    if pcp is None:
        pcp = 0.0

    return tmp, pcp


def get_hourly_weather_features(service_key: str, dong: str, yyyymmdd: str) -> pd.DataFrame:
    day_1to7 = ymd_to_day_1to7(yyyymmdd)

    rows = []
    for hour_slot in range(1, 11):
        rep_time = HOUR_SLOT_TO_REP_TIME[hour_slot]
        temp, rain = get_temp_rain_for_datetime(service_key, dong, yyyymmdd, rep_time)
        rows.append({
            "TA_YMD": int(yyyymmdd),
            "DONG": dong,
            "HOUR": hour_slot,
            "DAY": day_1to7,
            "TEMP": float(temp),
            "RAIN": float(rain),
        })
    return pd.DataFrame(rows)


# =========================
# E) 모델 학습 (시간대별 저장)
# =========================
def build_preprocess():
    cat_cols = ["DONG"]
    num_cols = ["DAY", "TEMP", "RAIN"]
    pre = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("num", "passthrough", num_cols),
        ],
        remainder="drop",
    )
    return pre


def train_and_save_models(random_state: int = 42) -> pd.DataFrame:
    df = pd.read_csv(DATA_CSV)

    df["DONG"] = df["DONG"].astype(str)
    df["HOUR"] = df["HOUR"].astype(int)
    df["DAY"] = df["DAY"].astype(int)
    df["TEMP"] = df["TEMP"].astype(float)
    df["RAIN"] = df["RAIN"].astype(float)

    y_cols = ["AMT", "CNT"]
    metrics = []

    for h in range(1, 11):
        sub = df[df["HOUR"] == h].copy()
        sub = sub.dropna(subset=["DONG", "DAY", "TEMP", "RAIN", "AMT", "CNT"])

        X = sub[["DONG", "DAY", "TEMP", "RAIN"]]
        y = sub[y_cols]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=random_state, shuffle=True
        )

        pre = build_preprocess()

        if h == 1:
            base_model = MLPRegressor(
                hidden_layer_sizes=(128, 64),
                activation="relu",
                solver="adam",
                alpha=1e-4,
                batch_size=256,
                learning_rate_init=1e-3,
                max_iter=300,
                random_state=42,
                early_stopping=True,
                n_iter_no_change=10,
                verbose=False,
            )
        else:
            base_model = XGBRegressor(
                n_estimators=600,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.9,
                colsample_bytree=0.9,
                reg_alpha=0.0,
                reg_lambda=1.0,
                objective="reg:squarederror",
                random_state=random_state,
                tree_method="hist",
                n_jobs=-1,
            )

        model = MultiOutputRegressor(base_model)

        pipe = Pipeline([
            ("pre", pre),
            ("model", model),
        ])

        pipe.fit(X_train, y_train)

        pred = pipe.predict(X_test)
        mae_amt = mean_absolute_error(y_test["AMT"].values, pred[:, 0])
        mae_cnt = mean_absolute_error(y_test["CNT"].values, pred[:, 1])

        model_path = os.path.join(MODEL_DIR, f"hour_{h:02d}_amt_cnt.joblib")
        joblib.dump(pipe, model_path)

        metrics.append({
            "HOUR": h,
            "MAE_AMT": mae_amt,
            "MAE_CNT": mae_cnt,
            "model_path": model_path,
            "model_type": "MLP(Deep)" if h == 1 else "XGB",
            "n_train": len(X_train),
            "n_test": len(X_test),
        })

    metrics_df = pd.DataFrame(metrics).sort_values("HOUR")
    metrics_df.to_csv(os.path.join(MODEL_DIR, "metrics_summary.csv"), index=False, encoding="utf-8-sig")

    # ✅ 너가 원한 출력 1) 유지
    print("metrics_summary.csv 저장 완료", flush=True)

    return metrics_df


# =========================
# F) 예측 (날짜+동 입력 -> 시간대별 AMT/CNT)
# =========================
def load_models():
    models = {}
    for h in range(1, 11):
        p = os.path.join(MODEL_DIR, f"hour_{h:02d}_amt_cnt.joblib")
        if not os.path.exists(p):
            raise FileNotFoundError(f"모델 파일이 없습니다: {p} (먼저 train_and_save_models() 실행)")
        models[h] = joblib.load(p)
    return models


def predict_day(yyyymmdd: str, dong: str, save_csv: bool = True) -> pd.DataFrame:
    feat_df = get_hourly_weather_features(SERVICE_KEY, dong, yyyymmdd)
    models = load_models()

    outs = []
    for h in range(1, 11):
        row = feat_df[feat_df["HOUR"] == h][["DONG", "DAY", "TEMP", "RAIN"]]
        pred_amt, pred_cnt = models[h].predict(row)[0]
        outs.append({
            "TA_YMD": yyyymmdd,
            "DONG": dong,
            "HOUR": h,
            "TEMP": float(feat_df.loc[feat_df["HOUR"] == h, "TEMP"].iloc[0]),
            "RAIN": float(feat_df.loc[feat_df["HOUR"] == h, "RAIN"].iloc[0]),
            "PRED_AMT": int(round(pred_amt)),
            "PRED_CNT": int(round(pred_cnt)),
        })

    pred_df = pd.DataFrame(outs)

    if save_csv:
        filename = f"pred_{dong}_{yyyymmdd}.csv"
        saved_path = os.path.join(OUTPUT_DIR, filename)
        pred_df.to_csv(saved_path, index=False, encoding="utf-8-sig")

        print(f"예측 결과 저장: {filename}", flush=True)
        # 만약 "전체 경로"도 보고싶으면 아래로 교체:
        # print(f"예측 결과 저장: {saved_path}", flush=True)

    return pred_df


# =========================
# G) 실행 예시 (원하는 3개만 출력)
# =========================
if __name__ == "__main__":
    _ = train_and_save_models()

    if not SERVICE_KEY:
        raise ValueError("환경변수 RAIN_ID(SERVICE_KEY)가 비었습니다. 예측을 하려면 .env를 확인하세요.")

    yyyymmdd = "20251230"
    dong = "매교동"
    pred_df = predict_day(yyyymmdd, dong, save_csv=True)

    print(pred_df, flush=True)


C:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


metrics_summary.csv 저장 완료
예측 결과 저장: pred_매교동_20251230.csv
     TA_YMD DONG  HOUR  TEMP  RAIN  PRED_AMT  PRED_CNT
0  20251230  매교동     1   0.0   0.0      4549         5
1  20251230  매교동     2   0.0   0.0    131480         5
2  20251230  매교동     3   0.0   0.0    223747        13
3  20251230  매교동     4   0.0   0.0    976706        75
4  20251230  매교동     5   0.0   0.0    999598        46
5  20251230  매교동     6   0.0   0.0   1306299        31
6  20251230  매교동     7   1.0   0.0    976212        50
7  20251230  매교동     8  -1.0   0.0   1759243        46
8  20251230  매교동     9  -1.0   0.0   1033210        21
9  20251230  매교동    10  -2.0   0.0    223855         5


In [18]:
if __name__ == "__main__":
    _ = train_and_save_models()

    if not SERVICE_KEY:
        raise ValueError("환경변수 RAIN_ID(SERVICE_KEY)가 비었습니다. 예측을 하려면 .env를 확인하세요.")

    yyyymmdd = "20251230"
    dong = "평동"
    pred_df = predict_day(yyyymmdd, dong, save_csv=True)

    print(pred_df, flush=True)

C:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


metrics_summary.csv 저장 완료
예측 결과 저장: pred_평동_20251230.csv
     TA_YMD DONG  HOUR  TEMP  RAIN  PRED_AMT  PRED_CNT
0  20251230   평동     1   0.0   0.0    758525        47
1  20251230   평동     2   0.0   0.0   1472177        55
2  20251230   평동     3   0.0   0.0   3691709       116
3  20251230   평동     4   0.0   0.0  15582414       720
4  20251230   평동     5   0.0   0.0  10255817       385
5  20251230   평동     6   0.0   0.0   4840355       193
6  20251230   평동     7   0.0   0.0   8269984       249
7  20251230   평동     8  -1.0   0.0  10334836       246
8  20251230   평동     9  -2.0   0.0   3836450        75
9  20251230   평동    10  -2.0   0.0    770741        17


# 흠..

In [4]:
import os
import joblib
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta

# =========================
# 설정
# =========================
from dotenv import load_dotenv
load_dotenv()

SERVICE_KEY = os.getenv("RAIN_ID")  # 기상청 단기예보 서비스키

MODELS_ROOT = "data/model"
BEST_MAP_PATH = os.path.join(MODELS_ROOT, "results_by_hour.csv")
BUNDLE_PATH_TEMPLATE = os.path.join(MODELS_ROOT, "HOUR_{hour:02d}", "best_model.joblib")

# ✅ AMT(매출) 학습 때 로그 타겟(y=log1p(AMT))이면 True
USE_LOG_TARGET = False

# (학습 때와 동일해야 함) 안전장치로 drop
DROP_COLS = ["UNIT", "DATE", "AMT", "CNT"]

# 격자 엑셀
GRID_XLSX_PATH = "data/수원시 격자.xlsx"
_GRID_CACHE = None  # 반복 로딩 방지 캐시

# =========================
# 항목요약 시간대(01~10)
# =========================
HOUR_BINS = {
    1: list(range(0, 7)),     # 00~06
    2: [7, 8],
    3: [9, 10],
    4: [11, 12],
    5: [13, 14],
    6: [15, 16],
    7: [17, 18],
    8: [19, 20],
    9: [21, 22],
    10: [23],
}

# 시간대 라벨(보기 좋게)
HOUR_LABELS = {
    1: "00~06",
    2: "07~08",
    3: "09~10",
    4: "11~12",
    5: "13~14",
    6: "15~16",
    7: "17~18",
    8: "19~20",
    9: "21~22",
    10: "23",
}

# =========================
# 날짜 파생 + DAY(01=월..07=일)
# =========================
def compute_day_code(date_yyyymmdd: str) -> int:
    dt = pd.to_datetime(str(date_yyyymmdd), format="%Y%m%d", errors="raise")
    return int(dt.weekday() + 1)

def make_date_features(date_yyyymmdd: str) -> dict:
    dt = pd.to_datetime(str(date_yyyymmdd), format="%Y%m%d", errors="raise")
    return {
        "TA_YMD": str(date_yyyymmdd),
        "YEAR": int(dt.year),
        "MONTH": int(dt.month),
        "DAY_OF_MONTH": int(dt.day),
        "WEEKOFYEAR": int(dt.isocalendar().week),
    }

# =========================
# 격자: DONG -> (nx, ny)
# =========================
def load_grid_table(path: str = GRID_XLSX_PATH) -> pd.DataFrame:
    """
    data/수원시 격자.xlsx
    - A열: 동이름, B열: nx, C열: ny (사용자 설명 기준)
    - header가 없다고 가정하고 A/B/C를 고정으로 읽음
      (만약 실제 파일에 헤더가 있으면 header=None -> header=0 으로 변경)
    """
    global _GRID_CACHE
    if _GRID_CACHE is not None:
        return _GRID_CACHE

    if not os.path.exists(path):
        raise FileNotFoundError(f"격자 파일이 없습니다: {path}")

    df = pd.read_excel(path, header=None)
    if df.shape[1] < 3:
        raise ValueError(f"격자 파일 컬럼 수가 부족합니다(A,B,C 필요): {path}")

    df = df.iloc[:, :3].copy()
    df.columns = ["DONG_NAME", "nx", "ny"]

    df["DONG_NAME"] = df["DONG_NAME"].astype(str).str.strip()
    df["nx"] = pd.to_numeric(df["nx"], errors="coerce")
    df["ny"] = pd.to_numeric(df["ny"], errors="coerce")
    df = df.dropna(subset=["nx", "ny"]).copy()
    df["nx"] = df["nx"].astype(int)
    df["ny"] = df["ny"].astype(int)

    _GRID_CACHE = df
    return df

def find_grid_by_dong(dong: str, path: str = GRID_XLSX_PATH) -> tuple[int, int]:
    """
    사용자가 입력한 dong(예: '행궁동', '수원시 팔달구 행궁동')으로
    엑셀 A열(DONG_NAME)에서 격자(nx,ny) 찾기.
    - 완전일치 우선, 없으면 포함검색(양방향)
    - 후보가 여러 개면 DONG_NAME 길이가 짧은 것을 우선 선택
    """
    dong = str(dong).strip()
    if not dong:
        raise ValueError("dong(동) 입력이 비었습니다.")

    grid = load_grid_table(path)

    # 1) 완전일치
    exact = grid[grid["DONG_NAME"] == dong]
    if not exact.empty:
        r = exact.iloc[0]
        return int(r["nx"]), int(r["ny"])

    # 2) 포함검색(양방향)
    a = grid["DONG_NAME"].str.contains(dong, na=False)
    b = grid["DONG_NAME"].apply(lambda x: str(dong).find(str(x)) >= 0)
    cand = grid[a | b].copy()

    # fallback: 마지막 토큰(보통 'OO동')로 재시도
    if cand.empty:
        tok = dong.split()[-1]
        cand = grid[grid["DONG_NAME"].str.contains(tok, na=False)].copy()

    if cand.empty:
        raise ValueError(f"'{dong}'에 해당하는 격자(nx,ny)를 찾지 못했습니다. 엑셀 A열 동이름을 확인하세요.")

    cand["name_len"] = cand["DONG_NAME"].str.len()
    cand = cand.sort_values(["name_len"]).reset_index(drop=True)
    r = cand.iloc[0]
    return int(r["nx"]), int(r["ny"])

# =========================
# 기상청 단기예보 TMP/PCP
# =========================
def _parse_pcp_to_mm(x):
    if x is None:
        return 0.0
    if isinstance(x, (int, float)):
        return float(x)
    s = str(x).strip()
    if s in ("강수없음", "없음", "", "nan", "NaN"):
        return 0.0
    if "미만" in s:
        try:
            num = float(s.replace("mm", "").replace("미만", "").strip())
            return max(0.0, num * 0.5)
        except:
            return 0.0
    if "~" in s:
        try:
            a, b = s.replace("mm", "").split("~")
            return (float(a) + float(b)) / 2.0
        except:
            return 0.0
    try:
        return float(s.replace("mm", ""))
    except:
        return 0.0

def fetch_vilage_fcst_json(service_key, base_date, base_time, nx, ny, num_rows=3000):
    url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    params = {
        "serviceKey": service_key,
        "pageNo": 1,
        "numOfRows": num_rows,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def fetch_hourly_tmp_pcp_for_date(service_key, target_yyyymmdd, nx, ny):
    base_times = ["2300", "2000", "1700", "1400", "1100", "0800", "0500", "0200"]
    now = datetime.now()
    today = now.strftime("%Y%m%d")
    yesterday = (now - timedelta(days=1)).strftime("%Y%m%d")

    last_err = None
    for bd in [today, yesterday]:
        for bt in base_times:
            try:
                js = fetch_vilage_fcst_json(service_key, bd, bt, nx, ny)
                items = js["response"]["body"]["items"]["item"]
                df = pd.DataFrame(items)

                df = df[df["category"].isin(["TMP", "PCP"])]
                df = df[df["fcstDate"].astype(str) == str(target_yyyymmdd)]
                if df.empty:
                    continue

                df["fcstTime"] = df["fcstTime"].astype(str).str.zfill(4)
                piv = df.pivot_table(
                    index="fcstTime",
                    columns="category",
                    values="fcstValue",
                    aggfunc="first"
                ).reset_index()

                if "TMP" not in piv.columns:
                    continue

                piv["TMP"] = piv["TMP"].astype(float)
                piv["PCP"] = piv["PCP"].apply(_parse_pcp_to_mm) if "PCP" in piv.columns else 0.0
                piv["PCP"] = piv["PCP"].astype(float)
                piv["base_used"] = f"{bd}-{bt}"
                return piv.sort_values("fcstTime").reset_index(drop=True)
            except Exception as e:
                last_err = e
                continue

    raise RuntimeError(f"해당 날짜의 기상청 예보 데이터를 찾지 못했습니다. last_err={last_err}")

def aggregate_weather_to_item_slots(hourly_df: pd.DataFrame) -> dict:
    hourly_df = hourly_df.copy()
    hourly_df["HOUR_OF_DAY"] = hourly_df["fcstTime"].str[:2].astype(int)

    out = {}
    for h, hours in HOUR_BINS.items():
        sub = hourly_df[hourly_df["HOUR_OF_DAY"].isin(hours)]
        out[h] = {
            "TEMP": float(sub["TMP"].mean()) if not sub.empty else np.nan,
            "RAIN": float(sub["PCP"].sum()) if not sub.empty else 0.0,
        }

    temps = [v["TEMP"] for v in out.values() if not pd.isna(v["TEMP"])]
    fill_temp = float(np.mean(temps)) if temps else 0.0
    for h in out:
        if pd.isna(out[h]["TEMP"]):
            out[h]["TEMP"] = fill_temp

    return out

# =========================
# (유틸) 동 입력 -> 해당 동의 기온 반환
# =========================
def get_temperature_by_dong(
    date_yyyymmdd: str,
    dong: str,
    slot_hour: int = 5,
    service_key: str = SERVICE_KEY
) -> float:
    if not service_key:
        raise ValueError("SERVICE_KEY(RAIN_ID)가 비어있습니다. .env 또는 환경변수를 확인하세요.")
    if not (1 <= int(slot_hour) <= 10):
        raise ValueError("slot_hour는 1~10 이어야 합니다.")

    nx, ny = find_grid_by_dong(dong)
    hourly_weather = fetch_hourly_tmp_pcp_for_date(service_key, date_yyyymmdd, nx, ny)
    weather_by_hour = aggregate_weather_to_item_slots(hourly_weather)

    temp = float(weather_by_hour[int(slot_hour)]["TEMP"])
    return round(temp, 1)

# =========================
# 모델 로드 + "모델이 기대하는 컬럼" 자동 맞춤
# =========================
def load_best_model_map():
    if not os.path.exists(BEST_MAP_PATH):
        raise FileNotFoundError(f"결과 파일이 없습니다: {BEST_MAP_PATH}")
    df = pd.read_csv(BEST_MAP_PATH)
    required_cols = {"HOUR", "BEST_MODEL"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"{BEST_MAP_PATH}에 필요한 컬럼이 없습니다. 필요: {required_cols}, 현재: {set(df.columns)}")
    return df.set_index("HOUR")["BEST_MODEL"].to_dict()

def load_best_bundle(hour: int):
    path = BUNDLE_PATH_TEMPLATE.format(hour=hour)
    if not os.path.exists(path):
        raise FileNotFoundError(f"번들 파일이 없습니다: {path}")
    bundle = joblib.load(path)
    if not isinstance(bundle, dict) or "preprocess" not in bundle or "model" not in bundle:
        raise ValueError(f"번들 포맷이 예상과 다릅니다: {path}")
    return bundle

def get_required_feature_names_from_preprocess(preprocess) -> list:
    names = []
    for _, _, cols in getattr(preprocess, "transformers_", []):
        if cols is None:
            continue
        if isinstance(cols, (list, tuple, np.ndarray, pd.Index)):
            names.extend(list(cols))
    return sorted(set([str(x) for x in names]))

def align_row_to_required_columns(X_row: pd.DataFrame, required_cols: list) -> pd.DataFrame:
    X = X_row.copy()

    if required_cols:
        drop_cols = [c for c in X.columns if c not in required_cols]
        if drop_cols:
            X = X.drop(columns=drop_cols, errors="ignore")

        for c in required_cols:
            if c not in X.columns:
                X[c] = np.nan

        X = X[required_cols]

    return X

def predict_with_bundle(bundle: dict, X_new: pd.DataFrame) -> np.ndarray:
    pre = bundle["preprocess"]
    model = bundle["model"]
    X_enc = pre.transform(X_new)
    return model.predict(X_enc)

# =========================
# ✅ 핵심: 날짜 1개 → 1~10시간대 "매출(AMT)" 예측
# =========================
def predict_sales_all_hours_with_weather(
    date_yyyymmdd: str,
    nx: int,
    ny: int,
    service_key: str = SERVICE_KEY
) -> pd.DataFrame:
    if not service_key:
        raise ValueError("SERVICE_KEY(RAIN_ID)가 비어있습니다. .env 또는 환경변수를 확인하세요.")

    day_code = compute_day_code(date_yyyymmdd)

    hourly_weather = fetch_hourly_tmp_pcp_for_date(service_key, date_yyyymmdd, nx, ny)
    weather_by_hour = aggregate_weather_to_item_slots(hourly_weather)

    best_map = load_best_model_map()
    rows = []

    for hour in range(1, 11):
        if hour not in best_map:
            continue

        bundle = load_best_bundle(hour)
        best_name = str(best_map[hour])

        row = {}
        row.update(make_date_features(date_yyyymmdd))
        row.update({
            "HOUR": int(hour),
            "DAY": int(day_code),
            "TEMP": float(weather_by_hour[hour]["TEMP"]),
            "RAIN": float(weather_by_hour[hour]["RAIN"]),
            "DELIV_TEMP": float(weather_by_hour[hour]["TEMP"]),
            "DELIV_RAIN": float(weather_by_hour[hour]["RAIN"]),
        })

        X_row = pd.DataFrame([row])
        X_row = X_row.drop(columns=[c for c in DROP_COLS if c in X_row.columns], errors="ignore")

        required = get_required_feature_names_from_preprocess(bundle["preprocess"])
        X_row_aligned = align_row_to_required_columns(X_row, required)

        pred = float(predict_with_bundle(bundle, X_row_aligned)[0])
        if USE_LOG_TARGET:
            pred = float(np.expm1(pred))

        pred = max(0.0, pred)  # 음수 방지

        rows.append({
            "TA_YMD": str(date_yyyymmdd),
            "DAY": int(day_code),
            "HOUR": int(hour),
            "HOUR_LABEL": HOUR_LABELS.get(int(hour), str(hour)),
            "BEST_MODEL": best_name,
            "TEMP": round(float(row["TEMP"]), 2),
            "RAIN": float(row["RAIN"]),
            "PRED_AMT": int(round(pred)),
            "KMA_BASE_USED": str(hourly_weather.loc[0, "base_used"]),
        })

    df = pd.DataFrame(rows).sort_values("HOUR").reset_index(drop=True)

    # ✅ (추가1) 하루 총매출(1~10 합)
    total_amt = int(df["PRED_AMT"].sum()) if not df.empty else 0
    df["PRED_AMT_TOTAL_DAY"] = total_amt  # 모든 행에 동일 값으로 들어감(조회 편의)

    return df

# =========================
# ✅ 핵심: 사용자 DONG 입력 버전
# =========================
def predict_sales_all_hours_with_weather_by_dong(
    date_yyyymmdd: str,
    dong: str,
    service_key: str = SERVICE_KEY
) -> pd.DataFrame:
    nx, ny = find_grid_by_dong(dong)
    df_pred = predict_sales_all_hours_with_weather(
        date_yyyymmdd=date_yyyymmdd,
        nx=nx,
        ny=ny,
        service_key=service_key
    )
    df_pred.insert(1, "DONG", str(dong))
    return df_pred

# =========================
# ✅ (추가2) 보기 좋게 출력하는 프린트 함수
# =========================
def print_sales_prediction(df_pred: pd.DataFrame) -> None:
    """
    predict_sales_all_hours_with_weather_* 결과(DataFrame)를
    시간대별 + 하루 총매출까지 보기 좋게 출력
    """
    if df_pred is None or df_pred.empty:
        print("예측 결과가 없습니다.")
        return

    date_yyyymmdd = str(df_pred.loc[0, "TA_YMD"])
    dong = str(df_pred.loc[0, "DONG"]) if "DONG" in df_pred.columns else "-"
    day_code = int(df_pred.loc[0, "DAY"])
    total_amt = int(df_pred.loc[0, "PRED_AMT_TOTAL_DAY"]) if "PRED_AMT_TOTAL_DAY" in df_pred.columns else int(df_pred["PRED_AMT"].sum())

    print("\n" + "=" * 80)
    print("📊 매출(AMT) 예측 결과")
    print("=" * 80)
    print(f"📍지역(동): {dong}")
    print(f"📅날짜: {date_yyyymmdd} (DAY코드={day_code})")
    print("-" * 80)

    # 시간대별 표 형태 출력
    # (시간대 / 기온 / 강수 / 예측매출)
    for _, r in df_pred.sort_values("HOUR").iterrows():
        hour = int(r["HOUR"])
        label = str(r.get("HOUR_LABEL", hour))
        temp = float(r.get("TEMP", np.nan))
        rain = float(r.get("RAIN", 0.0))
        pred_amt = int(r.get("PRED_AMT", 0))

        print(f"⏰ {hour:02d} ({label}) | 🌡️ {temp:5.1f}°C | 🌧️ {rain:6.1f}mm | 💰 {pred_amt:>10,} 원")

    print("-" * 80)
    print(f"✅ 하루 총매출(1~10합): 💰 {total_amt:>12,} 원")
    print("=" * 80 + "\n")

# =========================
# 사용 예시
# =========================
if __name__ == "__main__":
    df_pred = predict_sales_all_hours_with_weather_by_dong(
        date_yyyymmdd="20251231",
        dong="정자2동"
    )

    # DataFrame 출력
    print(df_pred)

    # 보기 좋게 콘솔 출력
    print_sales_prediction(df_pred)


     TA_YMD  DONG  DAY  HOUR HOUR_LABEL BEST_MODEL  TEMP  RAIN  PRED_AMT  \
0  20251231  정자2동    3     1      00~06        XGB -5.43   0.0     38545   
1  20251231  정자2동    3     2      07~08        XGB -8.00   0.0     11827   
2  20251231  정자2동    3     3      09~10        XGB -7.00   0.0     16934   
3  20251231  정자2동    3     4      11~12  EXTRATREE -5.50   0.0     25851   
4  20251231  정자2동    3     5      13~14  EXTRATREE -4.00   0.0     24799   
5  20251231  정자2동    3     6      15~16        XGB -4.00   0.0     20502   
6  20251231  정자2동    3     7      17~18  EXTRATREE -5.50   0.0     32290   
7  20251231  정자2동    3     8      19~20  EXTRATREE -6.50   0.0     38535   
8  20251231  정자2동    3     9      21~22        XGB -8.00   0.0     34702   
9  20251231  정자2동    3    10         23        XGB -9.00   0.0     51965   

   KMA_BASE_USED  PRED_AMT_TOTAL_DAY  
0  20251230-1100              295950  
1  20251230-1100              295950  
2  20251230-1100              295950  
3  2025